# Save Model Scikit-Learn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dataset = pd.read_csv('/content/drive/My Drive/BCML/iris.csv')

dataset.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
Y = dataset['Species'] # Target prediksi
x = dataset.drop(['Id', 'Species'], axis=1) # Data Id tidak diperlukan

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, Y, test_size=0.15)

In [5]:
# Save model dan scaler dengan pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression())])
pipeline.fit(x_train, y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_score, recall_score

## Modeling


In [7]:
from sklearn.metrics import f1_score, precision_score, recall_score
y_pred = pipeline.predict(x_test)

In [8]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(pipeline.score(x_test, y_test))) # .score() -> Accuracy
print(f1_score(y_test, y_pred, average='macro'))
print(precision_score(y_test, y_pred, average='macro'))
print(recall_score(y_test, y_pred, average='macro'))

Accuracy of logistic regression classifier on test set: 1.00
1.0
1.0
1.0


## Save model hasil training

In [9]:
import pickle

In [10]:
modelw = 'model.pkl'
pickle.dump(pipeline, open(modelw, 'wb'))

## Loading model hasil training

In [11]:
loaded_model = pickle.load(open(modelw, 'rb'))

In [12]:
y_pred_loaded = loaded_model.predict(x_test)

In [13]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(pipeline.score(x_test, y_test))) # .score() -> Accuracy
print(f1_score(y_test, y_pred_loaded, average='macro'))
print(precision_score(y_test, y_pred_loaded, average='macro'))
print(recall_score(y_test, y_pred_loaded, average='macro'))

Accuracy of logistic regression classifier on test set: 1.00
1.0
1.0
1.0


In [14]:
x_test.tail()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
52,6.9,3.1,4.9,1.5
10,5.4,3.7,1.5,0.2
129,7.2,3.0,5.8,1.6
148,6.2,3.4,5.4,2.3
94,5.6,2.7,4.2,1.3


## Test prediksi data tunggal

In [30]:

test_data = [[6.2,	3.4,	5.4,	2.3]]

prediction_example = loaded_model.predict(test_data)

prediction_example[0]

'Iris-virginica'

# Implementasi Ke Web Service

In [16]:
!pip install flask-ngrok

In [17]:
from flask_ngrok import run_with_ngrok
from flask import Flask

In [18]:
app = Flask(__name__) # Membuat sebuah object Flask

@app.route("/home")
def home():
    return "<h1>Running Flask on Google Colab!</h1>"

In [19]:
from flask import jsonify, request

In [20]:
@app.route('/classify', methods=['POST'])
def classification():
  sepal_width = float(request.json['sepal_length'])
  sepal_length = float(request.json['sepal_width'])
  petal_length = float(request.json['petal_length'])
  petal_width = float(request.json['petal_width'])

  modelw = pickle.load(open('model.pkl', 'rb'))

  data = [[sepal_length, sepal_width, petal_length, petal_width]]
  data = pipeline.predict(data)

  return jsonify({
      "output": data[0]
  })

In [21]:
run_with_ngrok(app)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9d9bc8d6c942.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Oct/2020 03:40:15] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2020 03:40:49] "POST /classify HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2020 03:40:56] "POST /classify HTTP/1.1" 200 -


## Melakukan prediksi melalui jupyter notebook

In [22]:
import requests
import json

Untuk melakukan prediksi di jupyter notebook, terlebih dahulu run model dan route app nya di notebook lain

In [28]:
url = 'http://3be9923cbef8.ngrok.io/classify'

data = {
	"sepal_width": 0.1,
	"sepal_length": 0.7,
	"petal_width": 0.7,
	"petal_length": 1.0
}

json_response = requests.post(url, json=data)

json_response
response = json.loads(json_response.text)

In [29]:
response['output']

'Iris-setosa'